## Evaluating a vectorial function on CPU and GPU

### CPU: plain and numpy

In [10]:
import numpy as np
from numba import njit, jit

# Python plain implementation w/ numba 
@njit
def grade2_vector(x, y, a, b, c):
    z = np.zeros(x.size)
    for i in range(x.size):
        z[i] = a*x[i]*x[i] + b*y[i] + c
    return z

# Numpy ufunc
def grade2_ufunc(x, y, a, b, c):
    return a*x**2 + b*y + c

# size of the vectors
size = 5_000_000

# allocating and populating the vectors
a_cpu = np.random.rand(size)
b_cpu = np.random.rand(size)
c_cpu = np.zeros(size)

a = 3.5
b = 2.8
c = 10

# Printing input values
#print(a_cpu)
#print(b_cpu)
# Random function in Numpy always use float64
print(a_cpu.dtype)

c_cpu = grade2_vector(a_cpu, b_cpu, a, b, c)


# Evaluating the time

# Numba Python: huge improvement, better that numpy code
%timeit -n 5 -r 2 grade2_vector(a_cpu, b_cpu, a, b, c)

# w/ a numpy ufunc manually coded
%timeit -n 5 -r 2 grade2_ufunc(a_cpu, b_cpu, a, b, c)

# using the general numpy ufunc 
%timeit -n 5 -r 2 a*a_cpu**2 + b*b_cpu + c



float64
8.14 ms ± 10.8 μs per loop (mean ± std. dev. of 2 runs, 5 loops each)
18.2 ms ± 76.9 μs per loop (mean ± std. dev. of 2 runs, 5 loops each)
18.1 ms ± 24.3 μs per loop (mean ± std. dev. of 2 runs, 5 loops each)


In [31]:
import cupy as cp
from cupyx.profiler import benchmark

#Paso a simple precisión (float32)
a_cpu_32 = a_cpu.astype(np.float32)
b_cpu_32 = b_cpu.astype(np.float32)

def cupy_copia(x, y, a, b, c):
    #Copia a GPU
    x_gpu = cp.asarray(x)
    y_gpu = cp.asarray(y)
    res_gpu = a * x_gpu**2 + b * y_gpu + c
    #Copia a CPU
    return cp.asnumpy(res_gpu)

print("Midiendo CuPy copiando los datos")
#Se usa benchmark de CuPy
print(benchmark(cupy_copia, (a_cpu_32, b_cpu_32, a, b, c), n_repeat=5))

#Sin copiar datos
#Creamos los arrays en la GPU
a_gpu_32 = cp.asarray(a_cpu_32)
b_gpu_32 = cp.asarray(b_cpu_32)

def cupy_sin_copia(x_gpu, y_gpu, a, b, c):
    # Cálculo directo en GPU
    return a * x_gpu**2 + b * y_gpu + c

print("\nMidiendo CuPy sin copiar los datos")
print(benchmark(cupy_sin_copia, (a_gpu_32, b_gpu_32, a, b, c), n_repeat=5))

Midiendo CuPy copiando los datos
cupy_copia          :    CPU:  7783.355 us   +/- 12.573 (min:  7762.952 / max:  7800.776) us     GPU-0:  7788.275 us   +/- 12.451 (min:  7767.712 / max:  7805.312) us

Midiendo CuPy sin copiar los datos
cupy_sin_copia      :    CPU:    99.027 us   +/-  0.997 (min:    97.396 / max:    99.957) us     GPU-0:   902.022 us   +/-  1.173 (min:   899.936 / max:   903.136) us


In [32]:
from numba import vectorize, float32, cuda

#ufunc para GPU
@vectorize(['float32(float32, float32, float32, float32, float32)'], target='cuda')
def grade2_numba_gpu(x, y, a, b, c):
    return a * x**2 + b * y + c

print("Midiendo Numba con copia")
%timeit -n 5 -r 2 grade2_numba_gpu(a_cpu_32, b_cpu_32, a, b, c)

#Se hace la copia
a_gpu = cuda.to_device(a_cpu_32)
b_gpu = cuda.to_device(b_cpu_32)

print("\nMidiendo Numba sin copia")
%timeit -n 5 -r 2 grade2_numba_gpu(a_gpu, b_gpu, a, b, c)

Midiendo Numba con copia
13.2 ms ± 3.56 ms per loop (mean ± std. dev. of 2 runs, 5 loops each)

Midiendo Numba sin copia
1.59 ms ± 101 μs per loop (mean ± std. dev. of 2 runs, 5 loops each)


### Resultados
Analizando los tiempos de ejecución, en primer lugar se observa que hay una gran diferencia entre ejecutar el código con copia (alrededor de 7.8 ms en CuPy) y sin copia (alrededor de 1.0 ms en CuPy). Esto demuestra que mover los datos de la memoria de la CPU a la GPU supone más tiempo que el cálculo matemático. Para arrays de este tamaño (5 millones de elementos), la transferencia de datos ocupa la gran mayoría del tiempo total.

En segundo lugar, respecto a la aceleración, se confirma que el cálculo en GPU (CuPy sin copia 1.0 ms) es mucho más rápido que la versión original de en CPU, demostando la utilidad de utilizar GPUs para operaciones de este tipo frente a la CPU.

Por último, al comparar las dos librerías utilizadas, en este caso específico CuPy (1.0 ms) ha resultado más eficiente que Numba (1.5 ms) en la ejecución sin copia, aunque ambas mejoran mucho el rendimiento con respecto a la ejecución en CPU.